In [1]:
# Używane biblioteki
import carla
from carla import Transform, Location, Rotation    # Potrzebne do przesunięcia punktu początkowego pojazdu
import random
import cv2                                         # Do wyświetlania obrazu
import math
import numpy as np
import csv
import time
import os
# Potrzebne  do storzenia trasy, w teście "Łosia"
import sys 
sys.path.append('C:/Users/Kacper/Desktop/Carla/WindowsNoEditor/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
#połączenie z symulacją 
client = carla.Client('localhost',2000)
world = client.get_world()

In [3]:
# Wybór mapy
client.set_timeout(60)
client.load_world('Town05')

In [4]:
#Wczytanie biblioteki z bluprintami 
bib = world.get_blueprint_library()

In [5]:
#Znalezienie punktów w których można tworzyć pojazdy
spawn_points = world.get_map().get_spawn_points()

In [6]:
#Stworzenie pojazdu 
pojazd_bib = bib.find('vehicle.lincoln.mkz_2020')
miejsce = spawn_points[8]
miejsce = Transform(Location(x=miejsce.location.x, y=miejsce.location.y + 20, z=miejsce.location.z), miejsce.rotation)

#miejsce = random.choice(spawn_points)
pojazd = world.try_spawn_actor(pojazd_bib, miejsce)
print(miejsce)

Transform(Location(x=35.056522, y=183.656631, z=0.300000), Rotation(pitch=0.000000, yaw=-89.977112, roll=0.000000))


In [7]:
# Uzyskanie danych fizycznych pojazdu
fizyka = pojazd.get_physics_control()
#Otrzymanie danych o kołach pojazdu
kola = fizyka.wheels
# Wyznacznie środka masy pojazdu
srodekMasy = fizyka.center_of_mass
lf = abs(miejsce.location.y - 0.01*kola[1].position.y) 
lr = abs(miejsce.location.y - 0.01*kola[3].position.y)

In [ ]:
#Pojazdy
#'vehicle.lincoln.mkz_2020'
#'vehicle.mercedes.coupe_2020'
#'vehicle.mercedes.sprinter'
#'vehicle.ford.ambulance'
#'vehicle.'volkswagen.t2_2021'
#'vehicle.mitsubishi.fusorosa'

# Tworzenie obiektu Location z wartościami x, y, z
miejsce_location = carla.Location(x=35.056522, y=163.656631, z=0.300000)

# Wypisanie wartości zmiennej
print(miejsce_location)

In [9]:
#Przeniesienie do lokazji utworzonego pojazdu
spectator = world.get_spectator()
transform = carla.Transform(pojazd.get_transform().transform(carla.Location(x=-4,z=2.5)),pojazd.get_transform().rotation) 
spectator.set_transform(transform) 

In [10]:
# Dodanie kamery
pozycja_z = 4 
pozycja_x = -10 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=pozycja_z,x=pozycja_x))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=pojazd)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}

camera.listen(lambda image: camera_callback(image,camera_data))

#Dodanie czujnika IMU

imu_bp = bib.find('sensor.other.imu')
imu_sens = world.spawn_actor(imu_bp, carla.Transform(), attach_to=pojazd)

imu_data = {'imu': None}

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }
    
imu_sens.listen(lambda data: imu_callback(data, imu_data))

In [12]:

en = 0.1
I = 0
e_pop = 0 

"""
Implementacja regulatora PID wykorzystywanego w tescie Łosia 

Wykorzystywana do sterowania prędkością pojazdu w celu ustalenia jej na zadanej wartości

y - obecna prędkość pojazdu

limMin - ograniczenie dolne sterowania pojazdem (minimalna wartość jaką możę zwrócić funkcja PID, min = 0)

limMax - ograniczenie górny sterowana pojazdem (maksymalna wartość jaką może zwórcić funkcja PID, maks = 1)

zadana - zadana prędkość którą chcemy osiągnąć

Nastawy otrzymane przy użyciu metody Ziglera-Nicholsa

"""   
def pid(y, limMin, limMax, zadana):
    global I
    global e_pop
    iMax = 0.45 #   0.45 :  0.4
    iMin = -0.45 # -0.45 : -0.5
    h = 1
    kp = 0.25  #3.366  #32.4        # 0.35 : 0.35 : 50 : 54  : 0.25
    Ti =  0.1  #6.16   #0.30955   # 4    : 0.1  : 0.01 : 0.2  : 0.1
    Td =  0.2  #3.08   #0.0773875 # 0.5  : 0.5  : 0    : 0.5  : 0.2

    e = zadana - y
    P = kp*e
    
    I = I + (h*kp/Ti) * e

    D = (Td/h) * (e - e_pop) 
    
    e_pop = e

    sterowanie = P + I + D
    if sterowanie>limMax:
        sterowanie = limMax
        I = I - (h*kp/Ti) * e
    elif sterowanie<limMin:
        sterowanie = limMin
        I = I - (h*kp/Ti) * e
    return sterowanie


"""" Poniżej dwie funkcje, które odpowiadaja za oblicznie kąta o jaki musi skręcić pojazd, żeby przejść do następnego punktu """
def kat_pomiedzy(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))
def znajdz_kat (auto, punkt): 

    pozycja_auta = auto.get_transform()
    auto_x = pozycja_auta.location.x
    auto_y = pozycja_auta.location.y
    punkt_x = punkt.transform.location.x
    punkt_y = punkt.transform.location.y

    x = (punkt_x - auto_x) / ((punkt_y - auto_y)**2 + (punkt_x -auto_x)**2)**0.5
    y = (punkt_y - auto_y) / ((punkt_y - auto_y)**2 + (punkt_x -auto_x)**2)**0.5

    auto_v = pozycja_auta.get_forward_vector()
    #kat = math.degrees(np.arctan2((y,x)) - np.arctan2(auto_v.y,auto_v.x)) 
    kat = kat_pomiedzy((x, y),(auto_v.x, auto_v.y))
    return kat


In [13]:
#Stworzenie pliku CSV
csv_filename = 'predkosci.csv'

# Nagłówki dla pliku CSV
csv_headers = ['Czas', 'Predkosc (km/h)']

# Otwórz plik CSV w trybie zapisu i zapisz nagłówki
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)

In [14]:
#Tworzenie Trasy 

globRoutePlan = GlobalRoutePlanner(world.get_map(),1)

miejsce_startowe = miejsce.location
miejsce_a = carla.Location(x=35.056522, y=75.0, z=0.300000)

# Linia prosta na której się rozpędzamy
droga1 = globRoutePlan.trace_route(miejsce_startowe,miejsce_a)

# Skręt w Lewo
miejsce_b = carla.Location(x=31.3, y=72.7, z=0.300000)
droga2 = globRoutePlan.trace_route(miejsce_a,miejsce_b)

    
#Odbicie w prawo
miejsce_c = carla.Location(x=31.3, y=69.0, z=0.300000)
droga3 = globRoutePlan.trace_route(miejsce_b,miejsce_c)


#Powrót na prawy pas
miejsce_d =carla.Location(x=35.056522, y=68.5, z=0.300000)
droga4 = globRoutePlan.trace_route(miejsce_c,miejsce_d)

miejsce_e = carla.Location(x=35.056522, y=-50.0, z=0.300000)
droga5 = globRoutePlan.trace_route(miejsce_d,miejsce_e)

droga = droga1 + droga2 + droga3 + droga4 + droga5

for punkt in droga:
    world.debug.draw_string(punkt[0].transform.location, '^', 
                            draw_shadow=False, color=carla.Color(r=255,g=0,b=0),
                            life_time=30.0, persistent_lines=True)


In [15]:
#Główny program

czas = 0  # Zmienna wykorzystywana w celu uzyskania pewnej ilości czasu skrętu
skret = 0 # Zmienna reprezentująca skręt pojazdu (Koła w lewo(-) lub w prawo(+)) 
flaga = 0 # Wykorzystywana w określeniu czy skręcamy w lewo czy w prawo (0 lewo)
osiag = 0 # Zmienna określająca, czy uzyskaliśmy odpowiednią prędkość  

font = cv2.FONT_HERSHEY_SIMPLEX
org2 = (30, 50)
org_compass = (30, 70) 
org_accel = (30, 90)
org_gyro = (30,120) 
fontScale = 0.5
color = (255, 255, 255)
grubosc = 1

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
status = 1
obecnyPunkt = 5
while obecnyPunkt<len(droga)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'): # przy wciśnięciu q wychodzimy z pętli symulacji
        break
    image = camera_data['image']
    
    
    while obecnyPunkt<len(droga) and pojazd.get_transform().location.distance(droga[obecnyPunkt][0].transform.location)<5:
            obecnyPunkt = obecnyPunkt + 1
        
    
    
    katSkretu = znajdz_kat(pojazd,droga[obecnyPunkt][0])
    v = pojazd.get_velocity() 
    # Obliczanie prędkości na podstawie wektorów pręskości w 3 wymiarach
    s = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
    # Otrzymanie parametru Roll przechyhlenia
    bodyRoll = pojazd.get_transform().rotation
    bodyRoll = bodyRoll.roll

    #Otrzymanie pozycji pojazdu
    
    current_time = time.time()
  
    
    # Zapis do pliku CSV
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([current_time, s ])
    image = cv2.putText(image,'Predkosc: '+str(int(s))+' km/h',org2, font, fontScale,color,grubosc,cv2.LINE_AA)
    image = cv2.putText(image, f'Przechylenie: {bodyRoll:.2f} stopni', (30, 180), font, fontScale, color, grubosc, cv2.LINE_AA)

      # Wyświetlanie odczytu z kompasu na obrazie
    # Wyświetlanie odczytu z kompasu na obrazie
    if imu_data['imu'] is not None:
        compass_heading = imu_data['imu']['compass']
        image = cv2.putText(image, f'Kompas: {compass_heading:.2f}', org_compass, font, fontScale, color, grubosc, cv2.LINE_AA)
    
        accel_heading = imu_data['imu']['accel']
        image = cv2.putText(image, f'Accel: x={accel_heading.x:.2f}, y={accel_heading.y:.2f}, z={accel_heading.z:.2f}', org_accel, font, fontScale, color, grubosc, cv2.LINE_AA)

    
    if katSkretu < -300:
         katSkretu = katSkretu + 360
    elif katSkretu > 300:
         katSkretu = katSkretu -360
    skret = katSkretu
    if katSkretu < -40:
         skret = -40
    elif katSkretu > 40:
         skret = 40
    skret = skret/75

    gaz  = pid(s, 0.25, 1, 50)
    pojazd.apply_control(carla.VehicleControl(throttle=gaz, steer=skret))
    cv2.imshow('RGB Camera',image)



imu_sens.stop()
imu_sens.destroy()
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


In [ ]:
""""

KOD SŁUŻĄCY DO PONOWNEGO URUCHAMINA TESTU

"""
maxRoll = 0
minRoll =0
#Stworzenie pojazdu 
pojazd_bib = bib.find('vehicle.lincoln.mkz_2020')
pojazd = world.try_spawn_actor(pojazd_bib, miejsce)
#print(miejsce)


# Uzyskanie danych fizycznych pojazdu
fizyka = pojazd.get_physics_control()
#Otrzymanie danych o kołach pojazdu
kola = fizyka.wheels
# Wyznacznie środka masy pojazdu
srodekMasy = fizyka.center_of_mass
lf = abs(miejsce.location.y - 0.01*kola[1].position.y) 
lr = abs(miejsce.location.y - 0.01*kola[3].position.y)

masa = fizyka.mass
print(masa)

# Dodanie kamery
pozycja_z = 4 
pozycja_x = -10 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=pozycja_z,x=pozycja_x))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=pojazd)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}

camera.listen(lambda image: camera_callback(image,camera_data))

#Dodanie czujnika IMU

imu_bp = bib.find('sensor.other.imu')
imu_sens = world.spawn_actor(imu_bp, carla.Transform(), attach_to=pojazd)

imu_data = {'imu': None}

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }
    
imu_sens.listen(lambda data: imu_callback(data, imu_data))

#Nazwy plików CSV i tworzenie

csv_filename = 'przechylenie.csv'


# Nagłówki dla pliku CSV dla pomiaru prędkości i przechylenia
csv_headers = ['Czas', 'Predkosc ','Przechylenie','x','y','Bf','Br']


if os.path.exists(csv_filename):
    os.remove(csv_filename)

# Otwarcie plików CSV w trybie zapisu i zapis nagłówków
try:
    with open(csv_filename, mode='w', newline='') as file:
     writer = csv.writer(file)
     writer.writerow(csv_headers)
except PermissionError as e:
    print("Zapis sie nie powiodl")

    
#Tworzenie Trasy

for punkt in droga:
    world.debug.draw_string(punkt[0].transform.location, '^', 
                            draw_shadow=False, color=carla.Color(r=255,g=0,b=0),
                            life_time=30.0, persistent_lines=True)

#Główny program

czas = 0  # Zmienna wykorzystywana w celu uzyskania pewnej ilości czasu skrętu
skret = 0 # Zmienna reprezentująca skręt pojazdu (Koła w lewo(-) lub w prawo(+))  

font = cv2.FONT_HERSHEY_SIMPLEX
org2 = (30, 50)
org_compass = (30, 70) 
org_accel = (30, 90)
org_gyro = (30,120) 
fontScale = 0.5
color = (255, 255, 255)
grubosc = 1

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
status = 0
obecnyPunkt = 5
pojazd.show_debug_telemetry()



while obecnyPunkt<len(droga)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'): # przy wciśnięciu q wychodzimy z pętli symulacji
        break
    image = camera_data['image']
    
    
    while obecnyPunkt<len(droga) and pojazd.get_transform().location.distance(droga[obecnyPunkt][0].transform.location)<5:
            obecnyPunkt = obecnyPunkt + 1
        
    
    katSkretu = znajdz_kat(pojazd,droga[obecnyPunkt][0])
    v = pojazd.get_velocity() 
    # Obliczanie prędkości na podstawie wektorów pręskości w 3 wymiarach
    s = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0) 
    
    current_time = time.time()
 
    
    # Otrzymanie parametru Roll przechyhlenia
    bodyRoll = pojazd.get_transform().rotation
    yaw = bodyRoll.yaw
    bodyRoll = bodyRoll.roll
    
    if bodyRoll > maxRoll:
        maxRoll = bodyRoll
    
    if bodyRoll < minRoll:
        minRoll = bodyRoll
    
    Bf = 0 
    Br = 0
    if status == 1 and abs(v.y) >0.0001:
        dYaw = yaw - yawPoprzednie
        dt = current_time - czasPoprzedni
        YawP = dYaw/dt 

        Bf = math.atan((v.x + lf*YawP)/(v.y)) - math.radians(katSkretu) #tire slip przednie

        Br = math.atan((v.x - lr*YawP)/(v.y))        #tire slip tylnie 
    yawPoprzednie = yaw
    czasPoprzedni = current_time
    status = 1

    #Otrzymanie pozycji pojazdu
    transform = pojazd.get_transform()
    x = transform.location.x
    y = transform.location.y
       
        
    # Zapis do plików CSV
    try:    
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([current_time, s ,bodyRoll,x,y,Bf,Br])
    except PermissionError as e:
        print("Błąd: {e}")
    except Exception as e:
        print("Błąd: {e}")
  
        
        
    image = cv2.putText(image,'Predkosc: '+str(int(s))+' km/h',org2, font, fontScale,color,grubosc,cv2.LINE_AA)
    image = cv2.putText(image, f'Przechylenie: {bodyRoll:.2f} stopni', (30, 180), font, fontScale, color, grubosc, cv2.LINE_AA)

      # Wyświetlanie odczytu z kompasu na obrazie
    # Wyświetlanie odczytu z kompasu na obrazie
    if imu_data['imu'] is not None:
        compass_heading = imu_data['imu']['compass']
        image = cv2.putText(image, f'Kompas: {compass_heading:.2f}', org_compass, font, fontScale, color, grubosc, cv2.LINE_AA)
    
        accel_heading = imu_data['imu']['accel']
        image = cv2.putText(image, f'Accel: x={accel_heading.x:.2f}, y={accel_heading.y:.2f}, z={accel_heading.z:.2f}', org_accel, font, fontScale, color, grubosc, cv2.LINE_AA)
    
    
    
    if katSkretu < -300:
         katSkretu = katSkretu + 360
    elif katSkretu > 300:
         katSkretu = katSkretu -360
    skret = katSkretu
    if katSkretu < -40:
         skret = -40
    elif katSkretu > 40:
         skret = 40
    skret = skret/75

    
    gaz  = pid(s, 0.4, 1.0, 75)  # 0.33, 0.8 : 0.25, 0.75
    
    pojazd.apply_control(carla.VehicleControl(throttle=gaz, steer=skret))
    cv2.imshow('RGB Camera',image)

#clean up

imu_sens.stop()
imu_sens.destroy()
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()   

print(maxRoll)
print(minRoll)


In [ ]:
#Pojazdy
#'vehicle.lincoln.mkz_2020'
#'vehicle.mercedes.coupe_2020'
#'vehicle.mercedes.sprinter'
#'vehicle.ford.ambulance'
#'vehicle.'volkswagen.t2_2021'w
#'vehicle.mitsubishi.fusorosa'
#'vehicle.tesla.cybertruck'
#vehicle.nissan.micra
#'vehicle.mini.cooper_s_2021'
#'vehicle.tesla.model3'
#'vehicle.ford.mustang'